In [1]:
import sys
import os
import pandas as pd
sys.path.append('/afs/cern.ch/cms/PPD/PdmV/tools/McM/')
from rest import McM

In [9]:
mcm = McM(dev=False)
time_per_event = {
    '20221220_ciperez_UnparToGG2016': 40,
    '20230103_ciperez_ADDToGGTuneCUEP8M1_16': 50,
    '20230103_ciperez_UnparToGGTuneCUEP8M1_16': 70,
}
tags = '20221220_ciperez_UnparToGG2016'
results = mcm.get('requests', query=f'tags={tags}&approval=none')
len(results)

[2023-01-08 04:55:56,177][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt


12

In [21]:
CLEAR=False
if CLEAR:
    for r in results:
        mcm.delete(object_type='requests', object_id=r['_id'])

In [10]:
results[0]['time_event']

[20]

In [11]:
for r in results:
    r['time_event'] = [70]
    r['size_event'] = [300]
    #r['fragment'] = r['fragment'].replace('from  Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *', 'from Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *')

In [5]:
results[0]

{'total_events': 100000,
 'config_id': [],
 'events_per_lumi': 0,
 'cmssw_release': 'CMSSW_7_1_47',
 'mcdb_id': -1,
 'transient_output_modules': [[]],
 'sequences': [{'eventcontent': ['RAWSIM'],
   'runsScenarioForMC': '',
   'donotDropOnInput': '',
   'extra': '',
   'slhc': '',
   'runsAndWeightsForMC': '',
   'outputCommand': '',
   'dropDescendant': False,
   'pileup': '',
   'hltProcess': '',
   'index': -1,
   'customise': 'SLHCUpgradeSimulations/Configuration/postLS1Customs.customisePostLS1',
   'custom_conditions': '',
   'datatier': ['GEN-SIM'],
   'processName': '',
   'inputCommands': '',
   'particle_table': '',
   'conditions': 'MCRUN2_71_V1::All',
   'harvesting': '',
   'triggerResultsProcess': '',
   'inputEventContent': '',
   'beamspot': 'Realistic50ns13TeVCollision',
   'restoreRNDSeeds': '',
   'filtername': '',
   'repacked': '',
   'customise_commands': '',
   'step': ['GEN', 'SIM'],
   'gflash': '',
   'procModifiers': '',
   'magField': '38T_PostLS1',
   'himix'

In [12]:
for r in results:
    output = mcm.update('requests', r)
    if output['results'] == False:
        print(r['_id'],output['message'])

In [8]:
time = set()
for r in results:
    time.add(r['time_event'][0])
time

{15}

{5}